In [1]:
# 90% CI from Monte Carlo
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:.2e}'.format)

# Data import
file_path = "/Users/elchulito/Library/CloudStorage/OneDrive-polymtlus/0 - A_Database and methodology_PhD/PlasticFADE.xlsx"
sheet_name = "Uncertainty"
data_CI = pd.read_excel(file_path, sheet_name=sheet_name, usecols="F:I", skiprows=1)
data_CI = data_CI.iloc[12:18] # Row index minus 3, change this range for other polymers
print(data_CI)

# Parameter estimates
a_i, delta_i, b_i, alpha_i, c_i, beta_i = data_CI.iloc[:, 2].values
# Standard deviations of parameters
a_i_std, delta_i_std, b_i_std, alpha_i_std, c_i_std, beta_i_std = data_CI.iloc[:, 3].values

            Process.1 Parameter.1  Estimate.1  Standard deviation.1
12  EPS fragmentation         a_i    5.66e-05              3.26e+00
13                NaN     delta_i    4.54e-05              1.66e+04
14                NaN         b_i    4.54e-05              5.84e+00
15                NaN     alpha_i    5.21e+00              7.35e-01
16                NaN         c_i    2.20e+04              2.90e+00
17                NaN      beta_i    1.33e+00              2.87e-01


In [5]:
# Input parameters
data_input = pd.read_excel(file_path, sheet_name="Results", usecols="A:E", skiprows=13)
data_input = data_input[data_input.iloc[:, 0] == "EPS"]  # Change for different polymer types
print(data_input)
data_input.columns = ['Polymer', 'Compartment', 's', 'I_j', 'P_j']

# Monte Carlo setup
N = 10000
np.random.seed(42)
results = []

# Loop through each row of input
for index, row in data_input.iterrows():
    s = row['s']
    I_j = row['I_j']
    P_j = row['P_j']

    # Monte Carlo with log-normal distribution
    a_i_samples = np.random.lognormal(np.log(a_i), a_i_std, N)
    delta_i_samples = np.random.lognormal(np.log(delta_i), delta_i_std, N)
    b_i_samples = np.random.lognormal(np.log(b_i), b_i_std, N)
    alpha_i_samples = np.random.lognormal(np.log(alpha_i), alpha_i_std, N)
    c_i_samples = np.random.lognormal(np.log(c_i), c_i_std, N)
    beta_i_samples = np.random.lognormal(np.log(beta_i), beta_i_std, N)

    # Impose caps to filter out values that are physically meaningless (n most empirical contexts, exponents above 4–5 are rare)
    delta_i_samples = np.clip(delta_i_samples, 0, 5)
    alpha_i_samples = np.clip(alpha_i_samples, 0, 5)
    beta_i_samples = np.clip(beta_i_samples, 0, 5)
    
    # Compute k_frag for each sample
    k_samples = a_i_samples * (s**delta_i_samples) * (b_i_samples * I_j**alpha_i_samples + c_i_samples * P_j**beta_i_samples)
    k_samples = k_samples[np.isfinite(k_samples)]  # Filter invalid samples (good habit, especially when NaNs are found in the CIs)

    # 90% CI in log-space
    if np.all(k_samples == 0):
        lower_bound, upper_bound = 0, 0
    else:
        log_k = np.log10(k_samples[k_samples > 0])  # Exclude zeros (for soil & sediment)
        lower_bound = 10 ** np.percentile(log_k, 5)
        upper_bound = 10 ** np.percentile(log_k, 95)
        
    k_point = a_i * (s**delta_i) * (b_i * I_j**alpha_i + c_i * P_j**beta_i)

    # If k_point is zero, force CI to be zero too
    if k_point == 0 or np.isnan(k_point):
        lower_bound = 0
        upper_bound = 0

    results.append({'Compartment': row['Compartment'], 'k_point': k_point, 'CI_lower': lower_bound, 'CI_upper': upper_bound})

# --- Display results ---
results_CI = pd.DataFrame(results)
print("\n", results_CI)
print(f"\n{N - len(k_samples)} out of {N} samples were invalid and removed.")

   Polymer (i) Compartment (j)  SA:V [cm-1]  I_j [W/m2]  P_j [mW]
12         EPS             Air           40    1.00e+01  3.17e-03
13         EPS            Soil           40    0.00e+00  0.00e+00
14         EPS           Beach           40    1.25e+01  4.70e-02
15         EPS   Water surface           40    1.00e+01  2.07e-02
16         EPS    Water column           40    0.00e+00  2.07e-08
17         EPS        Sediment           40    0.00e+00  0.00e+00

      Compartment  k_point  CI_lower  CI_upper
0            Air 9.97e-04  5.05e-06  1.91e+08
1           Soil 0.00e+00  0.00e+00  0.00e+00
2          Beach 2.25e-02  1.42e-04  1.49e+09
3  Water surface 7.50e-03  4.55e-05  6.56e+08
4   Water column 7.07e-11  3.12e-16  5.99e+01
5       Sediment 0.00e+00  0.00e+00  0.00e+00

0 out of 10000 samples were invalid and removed.


In [5]:
# Write confidence intervals back to Excel (without modifying)
import xlwings as xw

wb = xw.Book(file_path)  # file_path is your existing Excel file path
sheet = wb.sheets["Results"]

start_row = 27  # Change this index for other polymers
sheet.range(f'K{start_row}').options(index=False, header=False).value = results_CI['CI_lower'].values.reshape(-1, 1)
sheet.range(f'L{start_row}').options(index=False, header=False).value = results_CI['CI_upper'].values.reshape(-1, 1)

wb.save()
wb.close()